This code does the following:
1. Removes all events with no total yield
2. removes cows named "Unknown"
3. Sums all milkings for each cow in a single day and calculates number of milking events
4. Adds the specified weather data
5. Outputs one or multiple csv files with the combined data

Note that the file "AllMilkings.csv" is required to run this, as well as the relevant daily weather data

In [1]:
#Imports
import pandas as pd

In [2]:
def combineData(wCols, byFarm = True, total = True):
    #Loads the milking data
    milk = pd.read_csv("AllMilkings.csv", usecols=["FarmName_Pseudo", "SE_Number", "StartDate", "TotalYield"])
    #Removes som bad stuff
    milk = milk[milk["TotalYield"].notnull()]
    milk = milk[milk["SE_Number"] != "Unknown"]
    milk = milk.drop_duplicates()
    #Sorts by date
    milk = milk.sort_values(["StartDate"], ignore_index = True)

    #Creating series for sum and count for daily milk events
    milksum = milk.groupby(['SE_Number','StartDate'])["TotalYield"].sum()
    milkcount = milk.groupby(['SE_Number','StartDate'])["TotalYield"].count()
    #Remove old total yield
    milk = milk.drop(labels=["TotalYield"], axis = 1)
    #Drop "duplicates"
    milk = milk.drop_duplicates()
    #Setting multiindex and joining on multiindex
    milk = milk.set_index(["SE_Number","StartDate"])
    milk = milkcount.to_frame().rename({"TotalYield": "DailyCount"}, axis = 1).join(milk)
    milk = milksum.to_frame().rename({"TotalYield": "DailyYield"}, axis = 1).join(milk)
    #Resetting index
    milk = milk.reset_index(level=[0])
    milk = milk.sort_index()
    milk["DailyYield"] = round(milk["DailyYield"], 2)
    
    
    #Finds all unique farms in the milking data
    #Loads the weather data for said farms
    #Merges it with the milking data on date and farm
    uniqueFarms = milk["FarmName_Pseudo"].unique()
    df = pd.DataFrame()
    for farm in uniqueFarms:
        tf = pd.read_csv(f"{farm}_weather_daily.csv", usecols=wCols)
        tf["FarmName_Pseudo"] = farm
        df = pd.concat([df, tf])
    
    multimilk = pd.merge(milk, df,  how='left', left_on=['StartDate','FarmName_Pseudo'], right_on = ['Timestamp','FarmName_Pseudo'])
    
    if byFarm:
        for farm in uniqueFarms:
            ff = multimilk[multimilk["FarmName_Pseudo"] == farm]
            ff.to_csv(f"DailyMilk_{farm}_Weather.csv", index = False)
    
    if total:
        multimilk.to_csv("AllMilkingsDaily_Weather.csv", index = False)

In [6]:
#Choose which weather columns to add to the milking data
#"Timestamp" is required... might put in a check for it later
#Might be a good idea to add a description of the possible values
weatherCols = ["Timestamp", "T_C_mean", "THI_mean", "THI_adj_mean", "r_sfc_mean"]
combineData(weatherCols)